In [8]:
import numpy as np
import time

In [9]:
# ------------------ Levy Function Definition ------------------
def levy(x):
    w = 1 + (x - 1) / 4
    term1 = np.sin(np.pi * w[0])**2
    term3 = (w[-1] - 1)**2 * (1 + np.sin(2 * np.pi * w[-1])**2)
    term2 = np.sum((w[:-1] - 1)**2 * (1 + 10 * np.sin(np.pi * w[:-1] + 1)**2))
    return term1 + term2 + term3

In [10]:
# ------------------ DE Parameters ------------------
D = 50                       # Dimension
NP = 100                     # Population size
F = 0.5                      # Scaling factor
CR = 0.9                     # Crossover probability
max_generations = 1000
tol = 1e-5                   # Tolerance for convergence
lower_bound = -10
upper_bound = 10

In [11]:
# ------------------ Initialization ------------------
np.random.seed(42)
pop = np.random.uniform(low=lower_bound, high=upper_bound, size=(NP, D))
fitness = np.array([levy(ind) for ind in pop])

best_idx = np.argmin(fitness)
best_vector = pop[best_idx].copy()
best_fitness = fitness[best_idx]
fitness_history = [best_fitness]

start_time = time.time()


In [12]:
# ------------------ DE Main Loop ------------------
generation = 0
while generation < max_generations and best_fitness > tol:
    generation += 1

    for i in range(NP):
        idxs = list(range(NP))
        idxs.remove(i)
        r1, r2, r3 = np.random.choice(idxs, size=3, replace=False)
        donor = pop[r1] + F * (pop[r2] - pop[r3])
        donor = np.clip(donor, lower_bound, upper_bound)

        trial = np.empty(D)
        j_rand = np.random.randint(D)
        for j in range(D):
            if np.random.rand() < CR or j == j_rand:
                trial[j] = donor[j]
            else:
                trial[j] = pop[i][j]

        f_trial = levy(trial)
        if f_trial <= fitness[i]:
            pop[i] = trial
            fitness[i] = f_trial
            if f_trial < best_fitness:
                best_fitness = f_trial
                best_vector = trial.copy()

    fitness_history.append(best_fitness)

    if generation % 50 == 0 or best_fitness <= tol:
        print(f"Generation {generation:3d}: Best Fitness = {best_fitness:.6e}")

    if best_fitness <= tol:
        print(f"\nTerminated at generation {generation} (fitness ≤ {tol}).")
        break

end_time = time.time()

Generation  50: Best Fitness = 1.437421e+02
Generation 100: Best Fitness = 7.157399e+01
Generation 150: Best Fitness = 3.027256e+01
Generation 200: Best Fitness = 1.624117e+01
Generation 250: Best Fitness = 8.393261e+00
Generation 300: Best Fitness = 3.212858e+00
Generation 350: Best Fitness = 1.923403e+00
Generation 400: Best Fitness = 1.045177e+00
Generation 450: Best Fitness = 5.942329e-01
Generation 500: Best Fitness = 2.667132e-01
Generation 550: Best Fitness = 1.263164e-01
Generation 600: Best Fitness = 4.948503e-02
Generation 650: Best Fitness = 1.745200e-02
Generation 700: Best Fitness = 5.509170e-03
Generation 750: Best Fitness = 2.158324e-03
Generation 800: Best Fitness = 5.123345e-04
Generation 850: Best Fitness = 2.470252e-04
Generation 900: Best Fitness = 9.626552e-05
Generation 950: Best Fitness = 3.831019e-05
Generation 1000: Best Fitness = 1.628324e-05


In [13]:
# ------------------ Final Output ------------------
print("\nFinal Result:")
print(f"Best Vector: {best_vector}")
print(f"Best Fitness: {best_fitness}")
print(f"Total Time Taken: {end_time - start_time:.4f} seconds")


Final Result:
Best Vector: [0.9997775  0.99950805 0.99936893 0.99910739 1.0021833  0.99974685
 0.99945204 0.99969022 0.99998559 1.00158144 0.99969253 1.00115449
 0.99852779 0.99967001 1.00032484 1.00173142 0.99982387 1.00040529
 0.99908312 0.9997664  1.00051867 1.0003397  1.00016644 1.00140912
 0.99944982 0.99894061 0.99984756 1.00108874 0.99900827 1.00095162
 0.99940205 0.9998324  0.99859264 0.99928815 1.00017332 1.00115154
 0.99862515 1.00035984 1.00013763 0.9999267  0.99939648 0.99999178
 0.99953536 1.00011492 1.00019673 1.00084523 0.99966632 0.9995102
 1.00010241 0.99888523]
Best Fitness: 1.628323893462887e-05
Total Time Taken: 20.6056 seconds


In [14]:

import plotly.graph_objs as go

fig = go.Figure()
fig.add_trace(go.Scatter(
    y=fitness_history,
    x=list(range(len(fitness_history))),
    mode='lines+markers',
    name='Best Fitness',
    line=dict(color='royalblue'),
    marker=dict(size=4)
))

fig.update_layout(
    title='Differential Evolution on levy Function',
    xaxis_title='Generation',
    yaxis_title='Fitness Value',
    yaxis_type='log',
    template='plotly_white',
    showlegend=True
)

fig.show()
